# Filtering and slicing a WFS query

<a id='benchmark'></a>
## Benchmark your local performance
Depending on the speed of your connection, you may have already noticed that reading data from the web service can take some time.

So far you've been running Python in Jupyter on your own laptop. That means when you work on data from a web service you must first wait for it to download to your laptop. Also the speed and capacity of processing is limited by the speed and capacity of your laptop.

Let's time the `csv_read` command to see how long it takes. We can do that by noting the time just before the read and again just after, and subtracting one from the other. We do that with the `datetime` library. 

In [ ]:
import pandas
import datetime as dt

wfs_query = 'https://citydata.be.unsw.edu.au/geoserver/wfs?srsName=EPSG%3A4326&typename=geonode%3AInsideAirbnb_44_2015_17&outputFormat=csv&version=1.0.0&service=WFS&request=GetFeature'

maxFeatures_param = '&maxFeatures=10'
PropertyName_param = '&PropertyName=City,ScrapeDate,PropertyID,HostID,Latitude,Longitude,Price'
cql_filter_param = "&cql_filter=City='Sydney'+AND+ScrapeDate='2017-04-03T00:00:00'"

url = wfs_query + PropertyName_param + cql_filter_param #  + maxFeatures_param

before = dt.datetime.now()
listings = pandas.read_csv(url)
after = dt.datetime.now()
print('Read took:', after - before)

How long did it take? If you're on a fast connection and it only took a second or two, try using the `IN` operator to add more cities to your `cql_filter_param` just to get a benchmark for a slower transfer.

Here is the list of cities you found earlier:
```
['San Francisco', 'Sydney', 'Toronto', 'Vancouver', 'Venice',
       'Vienna', 'Washington, D.C.', 'Amsterdam', 'Antwerp', 'Austin',
       'Athens', 'Barcelona', 'Boston', 'Berlin', 'Brussels', 'Chicago',
       'London', 'Los Angeles', 'Madrid', 'Melbourne', 'Montreal',
       'New Orleans', 'New York City', 'Oakland', 'Paris', 'Portland',
       'Seattle', 'Trentino', 'Copenhagen', 'Dublin', 'Edinburgh',
       'Geneva', 'Hong Kong', 'Mallorca', 'Nashville', 'Quebec City',
       'San Diego', 'Victoria', 'Rome']
       ```
You'll also need to change the `ScrapeDate` to be anything in 2017, as those other cities may have been scraped on different date from the Sydney data. This can be accomplished with the `dateFormat` function as documented [here](http://docs.geoserver.org/stable/en/user/filter/function_reference.html#filter-function-reference).

In [ ]:
cql_filter_param = "&cql_filter=City+IN+('Sydney','Melbourne','London')+AND+dateFormat('y',ScrapeDate)='2017'"
# cql_filter_param = "&cql_filter=City+IN+('Sydney','Melbourne','London')"

url = wfs_query + PropertyName_param + cql_filter_param #  + maxFeatures_param

before = dt.datetime.now()
listings = pandas.read_csv(url)
after = dt.datetime.now()
print('Read took:', after - before)

Experiment with different combinations of cities and scrape dates to get a few benchmarks tim

Copy the code into new cells so you can keep each iteration and compare your times.

When done, open this workbook on your AWS server, and copy your code changes into it so you can run the same code again to compare the timing.

Do you see any difference? If not, perhaps you're on a very fast internet connection at Uni. Try this again some time from home!

In [ ]:
import pandas
wfs_query = 'https://citydata.be.unsw.edu.au/geoserver/wfs?srsName=EPSG%3A4326&typename=geonode%3AInsideAirbnb_44_2015_17&outputFormat=csv&version=1.0.0&service=WFS&request=GetFeature'
maxFeatures_param = '&maxFeatures=10'
url = wfs_query + maxFeatures_param
listings = pandas.read_csv(url)

Once you've finished comparing times, continue working on your AWS server (for its connection speed).

Now let's take a look at the data. We can list the columns:

In [ ]:
print(listings.columns)

Let's focus on just the columns that look interesting. Our code will be more efficient if we refine our WFS query to access only the columns we need. This will reduce the size of the WFS response transferred over the internet, and also reduce the amount of data Python needs to keep in memory.

We can do this by adding a filter. WFS filters are explained with examples by [Land Information New Zealand here](https://www.linz.govt.nz/data/linz-data-service/guides-and-documentation/wfs-filtering-by-attribute-or-feature).

We can refine our request to ask for just the City, ScrapeDate, PropertyID, HostID, Latitude, Longitude and Price. FID (feature identifier) will also be returned by default.

To do this, add a `PropertyNames` parameter to the request.

In [ ]:
PropertyName_param = '&PropertyName=City,ScrapeDate,PropertyID,HostID,Latitude,Longitude,Price'
url = wfs_query + maxFeatures_param + PropertyName_param
listings = pandas.read_csv(url)

listings

We need to remove the maxFeatures parameter so we are querying the full dataset, but if we do so, how much data will we get?

We can check the number of results by adding the parameter `resultType=hits` to our request (without it the default is `resultType=results`).

We also need to change the WFS version number from 1.0.0 to 1.1.0 because the resultType parameter was only introduced at this version of the [WFS spec](http://www.opengeospatial.org/standards/wfs).

<https://citydata.be.unsw.edu.au/geoserver/wfs?typename=geonode%3AInsideAirbnb_44_2015_17&version=1.1.0&service=WFS&request=GetFeature&resultType=hits>

Can you see the value for `numberOfFeatures` in the response to that URL? That's over a million records. Let's add a filter to look at just one city for now.

The WFS `filter` parameter is specified in XML, which is a bit tricky to write by hand, especially in a one-line URL.

Instead we'll use the non-standard [`cql_filter`](http://docs.geoserver.org/stable/en/user/tutorials/cql/cql_tutorial.html) parameter supported by Geoserver in which the filter is specified as a set of simple SQL-like constraints.

We'll filter to select just one city, but first we need to know what cities are in the dataset. Unfortunately WFS does not by default support a request to list unique values of an attribute. We need to get the cities from the full dataset, then find the unique list in Python.

In [ ]:
PropertyName_param = '&PropertyName=City'
url = wfs_query + PropertyName_param

cities = pandas.read_csv(url)

In [ ]:
cities["City"].unique()

Let's add the parameter `cql_filter=City="Sydney"` and count the number of records for Sydney:

<https://citydata.be.unsw.edu.au/geoserver/wfs?typename=geonode%3AInsideAirbnb_44_2015_17&version=1.1.0&service=WFS&request=GetFeature&cql_filter=City='Sydney'&resultType=hits>

That returned a more reasonable number of results.

So now let's put together our request, still with maxFeatures for now.

In [ ]:
PropertyName_param = '&PropertyName=City,ScrapeDate,PropertyID,HostID,Latitude,Longitude,Price'

cql_filter_param = "&cql_filter=City='Sydney'" # use double quotes because the filter contains single quotes

url = wfs_query + maxFeatures_param + PropertyName_param + cql_filter_param

listings = pandas.read_csv(url)

listings

The dataset is called *‘Inside Airbnb’ listings for 44 cities, **2015-17*** so we should check if we have duplicate records from different scrape dates. Let's find unique values for scrape date as we did for cities.

In [ ]:
PropertyName_param = '&PropertyName=ScrapeDate'

url = wfs_query + PropertyName_param + cql_filter_param
scrapedates = pandas.read_csv(url)

scrapedates.ScrapeDate.unique()

Indeed for this Sydney data there are three separate scrapes: in 2015, 2016 and 2017. Let's select just the 2017 records by specifying this in the `cql_filter` query parameter.

In [ ]:
PropertyName_param = '&PropertyName=City,ScrapeDate,PropertyID,HostID,Latitude,Longitude,Price'

cql_filter_param = "&cql_filter=City='Sydney'+AND+ScrapeDate='2017-04-03T00:00:00'"

url = wfs_query + maxFeatures_param + PropertyName_param + cql_filter_param

listings = pandas.read_csv(url)

listings

Looks good! Now we're ready to start visualising data with plots in the next workbook!